# Gif making function Simplex

In [18]:
import numpy as np
import plotly.graph_objects as go
import imageio
import os

def create_simplex_pivot_gif(vertices, vertices_visited, objective, dirname, gifname):
    """
    Creates a GIF showing the simplex algorithm's pivot steps over a polygonal feasible region.

    Parameters:
        vertices (array-like): Vertices of the polygon as (x, y) coordinates.
        vertices_visited (list): Indices of vertices in the desired visit order.
        objective (tuple): Objective function coefficients (c1, c2) for z = c1*x1 + c2*x2.
        dirname (str): Directory name to save frames.
        gifname (str): Name for the output GIF file (without extension).
    """
    c1, c2 = objective
    smooth_steps = 10  # Number of steps for smooth interpolation
    pause_frames = 5  # Number of frames to pause after drawing each arrow

    # Create directory for frames
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # Ensure vertices and visit order are NumPy arrays
    vertices = np.array(vertices)
    vertices_visited = np.array(vertices_visited)

    # Track visited edges
    path_edges = []
    images = []  # Initialize the images list for storing frames

    # Generate frames
    frame_count = 0
    for i in range(1, len(vertices_visited)):
        start_vertex = vertices[vertices_visited[i - 1]]
        end_vertex = vertices[vertices_visited[i]]
        path_edges.append((start_vertex, end_vertex))  # Save the current edge

        # Draw the arrow first (no green dot yet)
        fig = go.Figure()

        # Plot the feasible region
        fig.add_trace(go.Scatter(
            x=vertices[:, 0],
            y=vertices[:, 1],
            fill="toself",
            line=dict(color="blue", width=2),
            name="Feasible Region"
        ))

        # Draw cumulative path arrows
        for edge in path_edges:
            fig.add_annotation(
                x=edge[1][0], y=edge[1][1],
                ax=edge[0][0], ay=edge[0][1],
                xref="x", yref="y", axref="x", ayref="y",
                showarrow=True,
                arrowhead=3,
                arrowsize=1.5,
                arrowwidth=2,
                arrowcolor="orange"
            )

        # Add annotations for objective value and coordinates outside the plot
        current_z = c1 * end_vertex[0] + c2 * end_vertex[1]
        fig.add_annotation(
            xref="paper", yref="paper",
            x=1.05, y=0.8,
            text=f"Coordinates: ({end_vertex[0]:.2f}, {end_vertex[1]:.2f})",
            showarrow=False,
            font=dict(size=12)
        )
        fig.add_annotation(
            xref="paper", yref="paper",
            x=1.05, y=0.7,
            text=f"Objective: z = {current_z:.2f}",
            showarrow=False,
            font=dict(size=12)
        )

        # Update layout
        fig.update_layout(
            title=f"Simplex Pivot Step {i}: Drawing Arrow",
            xaxis=dict(range=[-1, np.max(vertices[:, 0]) + 1], title="x1"),
            yaxis=dict(range=[-1, np.max(vertices[:, 1]) + 1], title="x2"),
            showlegend=False
        )

        # Save the frame with the arrow
        frame_path = f"{dirname}/frame_{frame_count}.png"
        fig.write_image(frame_path)
        images.append(imageio.imread(frame_path))  # Add the frame to the images list
        frame_count += 1

        # Pause frames after drawing the arrow
        for _ in range(pause_frames):
            images.append(imageio.imread(frame_path))

        # Move the green dot smoothly to the next vertex
        for step in range(smooth_steps + 1):
            t = step / smooth_steps
            interp_vertex = (1 - t) * start_vertex + t * end_vertex

            # Create a new figure
            fig = go.Figure()

            # Plot the feasible region
            fig.add_trace(go.Scatter(
                x=vertices[:, 0],
                y=vertices[:, 1],
                fill="toself",
                line=dict(color="blue", width=2),
                name="Feasible Region"
            ))

            # Draw cumulative path arrows
            for edge in path_edges:
                fig.add_annotation(
                    x=edge[1][0], y=edge[1][1],
                    ax=edge[0][0], ay=edge[0][1],
                    xref="x", yref="y", axref="x", ayref="y",
                    showarrow=True,
                    arrowhead=3,
                    arrowsize=1.5,
                    arrowwidth=2,
                    arrowcolor="orange"
                )

            # Add the green dot (current vertex marker)
            fig.add_trace(go.Scatter(
                x=[interp_vertex[0]],
                y=[interp_vertex[1]],
                mode="markers",
                marker=dict(size=12, color="green"),
                name="Current Vertex"
            ))

            # Add annotations for objective value and coordinates outside the plot
            current_z = c1 * interp_vertex[0] + c2 * interp_vertex[1]
            fig.add_annotation(
                xref="paper", yref="paper",
                x=1.05, y=0.8,
                text=f"Coordinates: ({interp_vertex[0]:.2f}, {interp_vertex[1]:.2f})",
                showarrow=False,
                font=dict(size=12)
            )
            fig.add_annotation(
                xref="paper", yref="paper",
                x=1.05, y=0.7,
                text=f"Objective: z = {current_z:.2f}",
                showarrow=False,
                font=dict(size=12)
            )

            # Update layout
            fig.update_layout(
                title=f"Simplex Pivot Step {i}: Moving to Next Vertex",
                xaxis=dict(range=[-1, np.max(vertices[:, 0]) + 1], title="x1"),
                yaxis=dict(range=[-1, np.max(vertices[:, 1]) + 1], title="x2"),
                showlegend=False
            )

            # Save the frame with the moving green dot
            frame_path = f"{dirname}/frame_{frame_count}.png"
            fig.write_image(frame_path)
            images.append(imageio.imread(frame_path))  # Add the frame to the images list
            frame_count += 1

    # Add pause on the last frame
    for _ in range(10):  # Pause for 10 frames
        images.append(imageio.imread(f"{dirname}/frame_{frame_count - 1}.png"))

    # Create the GIF
    output_gif = f"{gifname}.gif"
    imageio.mimsave(output_gif, images, duration=0.2)  # 0.2s per frame

    print(f"Simplex pivot animation saved as {output_gif}")


# Make some examples

In [19]:
# Define the polygon vertices (feasible region)
polygon_vertices = [
    [0, 0],
    [3, 0],
    [4, 1],
    [2, 3],
    [0, 4],
    [0, 0]  # Closing the polygon
]

# Define the order in which vertices are visited (indices of polygon_vertices)
vertices_visited = [0, 1, 2, 3]

# Objective function coefficients (c1, c2)
objective = (2, 3)  # Example: 2*x1 + 3*x2

# Directory to save frames and GIF name
dirname = "simplex_frames"
gifname = "simplex_pivot_with_path"

# Call the function
create_simplex_pivot_gif(polygon_vertices, vertices_visited, objective, dirname, gifname)


Simplex pivot animation saved as simplex_pivot_with_path.gif


In [20]:
# Define the polygon vertices (feasible region)
polygon_vertices = [
    [0, 0],
    [3, 0],
    [4, 1],
    [2, 3],
    [0, 4],
    [0, 0]  # Closing the polygon
]

# Define the order in which vertices are visited (indices of polygon_vertices)
vertices_visited = [0, 4,  3]

# Objective function coefficients (c1, c2)
objective = (2, 3)  # Example: 2*x1 + 3*x2

# Directory to save frames and GIF name
dirname = "simplex_frames2"
gifname = "simplex_pivot_with_path2"

# Call the function
create_simplex_pivot_gif(polygon_vertices, vertices_visited, objective, dirname, gifname)


Simplex pivot animation saved as simplex_pivot_with_path2.gif
